In [1]:
from aimet_torch.cross_layer_equalization import equalize_model
from torchvision import models
import torch
import MobileNetV2Pretrained
import os
import sys

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(ROOT_DIR)

from utils.input_getter import mug_image_getter

# Load a pretrained model

image_getter = mug_image_getter()
mug_image = image_getter.get_input_torch()
print(mug_image.shape)

modelOriginal   = MobileNetV2Pretrained.PreprocessedMobileNetV2(64, r"model/mobilenet_v2_coco.pth")
modelQuantized  = MobileNetV2Pretrained.PreprocessedMobileNetV2(64, r"model/mobilenet_v2_coco.pth")

modelOriginal.eval()
modelQuantized.eval()

# model = models.resnet18(pretrained=True)
# model = torch.nn.Sequential(
#     torch.nn.Flatten(),
#     torch.nn.Linear(3 * 224 * 224, 1024),
#     torch.nn.ReLU(),
#     torch.nn.Linear(1024, 512),
#     torch.nn.ReLU(),
#     torch.nn.Linear(512, 256),
#     torch.nn.ReLU()
# )


print(modelQuantized)
print(modelOriginal)

2025-02-18 22:40:40,966 - root - INFO - AIMET


c:\Users\Pro\Documents\___FAKULTET___\LPCV\LPCV_2025_T1\LPCV_2025_T1\venv\Lib\site-packages\onnxscript\converter.py:823: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
<frozen abc>:106: FutureWarning: `NLLLoss2d` has been deprecated. Please use `NLLLoss` instead as a drop-in replacement and see https://pytorch.org/docs/main/nn.html#torch.nn.NLLLoss for more details.


torch.Size([1, 3, 224, 224])
PreprocessedMobileNetV2(
  (mobilenet_v2): MobileNetV2(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): Conv2dNormActivation(


In [2]:
from utils import helper
from dataset import utils as dsutils

helper.print_probablities_from_output(modelOriginal(mug_image), dsutils.GLOBAL_CLASSES)
helper.print_probablities_from_output(modelQuantized(mug_image), dsutils.GLOBAL_CLASSES)

# print(modelOriginal(mug_image))
# print(modelQuantized(mug_image))

Top-5 predictions for  on :
31 Cup                   78.5%
34 Bowl                   6.4%
37 Orange                 1.2%
33 Spoon                  1.2%
41 Donut                  0.9%
Top-5 predictions for  on :
31 Cup                   78.5%
34 Bowl                   6.4%
37 Orange                 1.2%
33 Spoon                  1.2%
41 Donut                  0.9%


In [3]:

param_num = 0

# print(list(model.parameters())[param_num])
# print(list(model.parameters())[param_num].shape)

# Apply Cross-Layer Equalization (CLE)
# This modifies the model in place

input_shape = (1, 3, 224, 224)
dummy_input = torch.randn(input_shape)

print("CLE Started")

equalize_model(modelQuantized, dummy_input = dummy_input)

print("Cross-Layer Equalization applied successfully!")

print(modelQuantized)

CLE Started
2025-02-18 22:40:51,953 - ConnectedGraph - WARNING - Unable to isolate model outputs.
Cross-Layer Equalization applied successfully!
PreprocessedMobileNetV2(
  (mobilenet_v2): MobileNetV2(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (1): Identity()
        (2): ReLU()
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32)
            (1): Identity()
            (2): ReLU()
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1))
          (2): Identity()
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1))
            (1): Identity()
            (2): ReLU()
          )
          (1): Conv2dN

In [4]:
helper.print_probablities_from_output(modelOriginal(mug_image), dsutils.GLOBAL_CLASSES)
helper.print_probablities_from_output(modelQuantized(mug_image), dsutils.GLOBAL_CLASSES)

Top-5 predictions for  on :
31 Cup                   78.5%
34 Bowl                   6.4%
37 Orange                 1.2%
33 Spoon                  1.2%
41 Donut                  0.9%
Top-5 predictions for  on :
31 Cup                   94.5%
34 Bowl                   1.8%
35 Banana                 0.4%
37 Orange                 0.3%
33 Spoon                  0.2%


In [ ]:
# Fix posle nemam pojma kako radi

# from aimet_torch.bias_correction import correct_bias
# import torch

# # Define your model (example: ResNet18)
# model = models.resnet18(pretrained=True)

# # Define a dummy dataset (batch size = 1, shape = [1, 3, 224, 224])
# dummy_input = torch.rand(1, 3, 224, 224)

# # Apply Bias Correction
# correct_bias(model)

# print("Bias Correction applied successfully!")

In [5]:
from aimet_torch.quantsim import QuantizationSimModel
from aimet_common.defs import QuantScheme
from torchvision import transforms
from torch.utils.data import DataLoader

# Dummy input to define the model input size
dummy_input = torch.rand(1, 3, 224, 224)

# Step 1: Create QuantizationSimModel
sim = QuantizationSimModel(modelQuantized, dummy_input=dummy_input,
                                     quant_scheme=QuantScheme.post_training_tf_enhanced,
                                     default_param_bw=8, default_output_bw=8)

from dataset import DatasetReader

datasetCOCO = DatasetReader.COCODataset(
    annotation_file = r"C:/Users/Pro/Documents/___FAKULTET___/LPCV/LPCV_2025_T1/data/annotations/annotations/instances_val2017.json", 
    image_dir= r'C:/Users/Pro/Documents/___FAKULTET___/LPCV/LPCV_2025_T1/data/val2017/val2017',
    target_classes=[s.lower() for s in dsutils.GLOBAL_CLASSES],
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]))

dataloader = DataLoader(datasetCOCO, batch_size=1, shuffle=True)

print(len(dataloader))


# Step 2: Compute Encodings (calibration)
def calibration_function(model, eval_iterations = 50, use_cuda = False):
    for i, data in enumerate(dataloader):
        image, target = data
        model(image)
        if (i > eval_iterations):
            break
    print(eval_iterations)
    print("cal")


2025-02-18 22:41:04,014 - ConnectedGraph - WARNING - Unable to isolate model outputs.
2025-02-18 22:41:04,087 - Quant - INFO - No config file provided, defaulting to config file at c:\Users\Pro\Documents\___FAKULTET___\LPCV\LPCV_2025_T1\LPCV_2025_T1\venv\Lib\site-packages\aimet_common\quantsim_config\default_config.json
2025-02-18 22:41:04,107 - Quant - INFO - Unsupported op type Squeeze
2025-02-18 22:41:04,108 - Quant - INFO - Unsupported op type Mean
2025-02-18 22:41:04,114 - Quant - INFO - Selecting DefaultOpInstanceConfigGenerator to compute the specialized config. hw_version:default
23043


In [6]:
print(sim.model)

PreprocessedMobileNetV2(
  (mobilenet_v2): MobileNetV2(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): QuantizedConv2d(
          3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)
          (param_quantizers): ModuleDict(
            (weight): QuantizeDequantize(shape=(), qmin=-128, qmax=127, symmetric=True)
            (bias): None
          )
          (input_quantizers): ModuleList(
            (0): QuantizeDequantize(shape=(), qmin=0, qmax=255, symmetric=False)
          )
          (output_quantizers): ModuleList(
            (0): None
          )
        )
        (1): Identity()
        (2): QuantizedReLU(
          (param_quantizers): ModuleDict()
          (input_quantizers): ModuleList(
            (0): None
          )
          (output_quantizers): ModuleList(
            (0): QuantizeDequantize(shape=(), qmin=0, qmax=255, symmetric=False)
          )
        )
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): 

In [ ]:
sim.compute_encodings(calibration_function)

In [ ]:
print(sim.model)

In [ ]:
sim.model(mug_image)

In [7]:

# # Step 3: Export Quantized Model
os.makedirs('quantized', exist_ok=True)
sim.export(path='quantized', filename_prefix='ref_solution_quantized', dummy_input=dummy_input)


2025-02-18 22:41:21,216 - Utils - INFO - successfully created onnx model with 88/100 node names updated
2025-02-18 22:41:21,294 - Quant - WARNING - The following layers were not found in the exported onnx model. Encodings for these layers will not appear in the exported encodings file, however it will continue to exist in torch encoding file:
['mobilenet_v2.classifier.0']
This can be due to several reasons:
	- The layer is set to quantize with float datatype, but was not exercised in compute encodings. Not an issue if the layer is not meant to be run.
	- The layer has valid encodings but was not seen while exporting to onnx using the dummy input provided in sim.export(). Ensure that the dummy input covers all layers.
2025-02-18 22:41:21,295 - Quant - INFO - Layers excluded from quantization: []
2025-02-18 22:41:21,296 - Quant - WARNING - Quantsim export will stop exporting encodings for saving and loading in a future AIMET release.
To export encodings for saving and loading, use Quanti

In [ ]:
print(sim.model)

In [ ]:
sim.model.mobilenet_v2.features[0][0]

In [ ]:
sim.model(mug_image)

In [ ]:
helper.print_probablities_from_output(sim.model(mug_image), dsutils.GLOBAL_CLASSES)